## Voice processor PoC

### Goal

Give back to Lucas free time by automatically processing his voice notes and turning them into feedbacks to kart drivers

### Estimated process

Inputs:

- Voice recorded feedbacks splitted by (1) race and (2) turn
- PDF with race results to identify drivers by their kart number

Processing:

- Create the driver/kart number/position record in memory
- Transform "raw" voice feedback into text and store it in the database
- Split the feedbacks by driver and turn

Outputs:

- Generate a nice feedback in text format to be sent via WhatsApp
- Generate a nice audio recorded feedback to be sent via WhatsApp


## Step 1

Process PDF

In [27]:
from dataclasses import dataclass

@dataclass
class Driver:
    position: int
    kart_number: int
    name: str


In [38]:
from pathlib import Path
import PyPDF2

# Define race name
race_name = "Tabajara 1"
drivers = []

# Find PDF file in data directory
race_dir = Path(f"data/input/{race_name}")
pdf_files = list(race_dir.glob("*.pdf"))

if not pdf_files:
    print(f"No PDF files found in {race_dir}")
else:
    pdf_path = pdf_files[0]

    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        first_page = reader.pages[0]
        text = first_page.extract_text().split("UF")[1]

        for line in text.split('MG'):
            if line.strip():
                parts = line.split(" ")

                if parts[0] == "\nMelhor":
                    break

                position = int(parts[0])
                kart_number = int(parts[1])
                driver_name = " ".join(parts[2:len(parts)-9]).replace("\n", "")
                driver = Driver(position=position, kart_number=kart_number, name=driver_name)
                drivers.append(driver)

print(drivers)


[Driver(position=1, kart_number=137, name='EMILIO DE SOUZA AMADEI BERIN...'), Driver(position=2, kart_number=144, name='JULIO RIBEIRO'), Driver(position=3, kart_number=164, name='THIAGO OLIVEIRA'), Driver(position=4, kart_number=107, name='Luis Caceres'), Driver(position=5, kart_number=143, name='EMILIO S A BERINGHS PAI'), Driver(position=6, kart_number=147, name='Guiga Guilherme Rodrigues'), Driver(position=7, kart_number=140, name='RODRIGO FERNANDES FARIA'), Driver(position=8, kart_number=126, name='Lucas Prates'), Driver(position=9, kart_number=156, name='Carlos Sampaio'), Driver(position=10, kart_number=142, name='ANDRÉ LA ROCCA'), Driver(position=11, kart_number=136, name='GERALDO MAGELA JACINTO'), Driver(position=12, kart_number=116, name='CLAUDEMIR SILVEIRA MAGNO'), Driver(position=13, kart_number=167, name='Newton Angelini')]
